Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [2]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
!pip install pandas_profiling

In [58]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [60]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [61]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [62]:
df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,23030,23031,23032,23033,23034,23035,23036,23037,23038,23039
BOROUGH,1,1,1,1,1,1,1,2,2,2,...,4,4,4,4,4,4,4,4,4,4
NEIGHBORHOOD,OTHER,OTHER,OTHER,OTHER,UPPER EAST SIDE (59-79),UPPER EAST SIDE (79-96),OTHER,OTHER,OTHER,OTHER,...,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER
BUILDING_CLASS_CATEGORY,13 CONDOS - ELEVATOR APARTMENTS,21 OFFICE BUILDINGS,21 OFFICE BUILDINGS,13 CONDOS - ELEVATOR APARTMENTS,15 CONDOS - 2-10 UNIT RESIDENTIAL,07 RENTALS - WALKUP APARTMENTS,07 RENTALS - WALKUP APARTMENTS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,...,02 TWO FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,02 TWO FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,09 COOPS - WALKUP APARTMENTS,10 COOPS - ELEVATOR APARTMENTS,02 TWO FAMILY DWELLINGS,12 CONDOS - WALKUP APARTMENTS
TAX_CLASS_AT_PRESENT,2,4,4,2,2C,2B,2B,1,1,1,...,1,1,1,1,1,1,2,2,1,2
BLOCK,716,812,839,592,1379,1551,1891,4090,4120,4120,...,10162,11612,11808,12295,12536,10965,169,131,8932,1216
LOT,1246,68,69,1041,1402,131,159,37,18,20,...,52,73,50,23,38,276,29,4,18,1161
EASE-MENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BUILDING_CLASS_AT_PRESENT,R4,O5,O5,R4,R1,C4,C4,A1,A5,A5,...,B1,A1,A0,A1,B3,A5,C6,D4,S2,R2
ADDRESS,"447 WEST 18TH STREET, PH12A",144 WEST 37TH STREET,40 WEST 38TH STREET,"1 SHERIDAN SQUARE, 8C","20 EAST 65TH STREET, B",354 EAST 89TH STREET,304 WEST 106 STREET,1193 SACKET AVENUE,1215 VAN NEST AVENUE,1211 VAN NEST AVENUE,...,104-59 164TH STREET,10919 132ND STREET,135-24 122ND STREET,134-34 157TH STREET,130-26 176 PLACE,111-17 FRANCIS LEWIS BLVD,"45-14 43RD STREET, 3C","50-05 43RD AVENUE, 3M",91-10 JAMAICA AVE,"61-05 39TH AVENUE, F5"
APARTMENT_NUMBER,PH12A,NaN,NaN,8C,B,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F5


In [63]:
df.isnull().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [9]:
import numpy as np

df = df.replace( '########',np.NaN, regex=True)

In [10]:
key = ['ZIP_CODE', 'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'SALE_PRICE']
replace= [-148.0,-147.0,-1.0]
df[key] = df[key].replace( replace,np.NaN, regex=True)

In [11]:
df.dtypes

BOROUGH                            object
NEIGHBORHOOD                       object
BUILDING_CLASS_CATEGORY            object
TAX_CLASS_AT_PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                         float64
BUILDING_CLASS_AT_PRESENT          object
ADDRESS                            object
APARTMENT_NUMBER                   object
ZIP_CODE                          float64
RESIDENTIAL_UNITS                 float64
COMMERCIAL_UNITS                  float64
TOTAL_UNITS                       float64
LAND_SQUARE_FEET                   object
GROSS_SQUARE_FEET                 float64
YEAR_BUILT                        float64
TAX_CLASS_AT_TIME_OF_SALE           int64
BUILDING_CLASS_AT_TIME_OF_SALE     object
SALE_PRICE                          int64
SALE_DATE                          object
dtype: object

In [12]:
df = df.drop(columns = ['APARTMENT_NUMBER', 'EASE-MENT'])

In [13]:
cols_mode = ['TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'ZIP_CODE', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS']

for column in cols_mode:
    df[column].fillna(df[column].mode()[0], inplace=True)

#'YEAR_BUILT' TOTAL_UNITS  GROSS_SQUARE_FEET LAND_SQUARE_FEET 

In [14]:
df.TOTAL_UNITS = df.RESIDENTIAL_UNITS + df.COMMERCIAL_UNITS

In [15]:
df['LAND_SQUARE_FEET'] = (
    df['LAND_SQUARE_FEET']
    .str.replace(',','')
)
df['LAND_SQUARE_FEET']= df['LAND_SQUARE_FEET'].astype('float64')

In [16]:
cols_mean = ['YEAR_BUILT', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET','LAND_SQUARE_FEET']

for column in cols_mean:
    df[column].fillna(df[column].mean(), inplace=True)

In [17]:
df.T.head(21)

,0,1,2,3,4,5,6,7,8,9,...,23030,23031,23032,23033,23034,23035,23036,23037,23038,23039
BOROUGH,1,1,1,1,1,1,1,2,2,2,...,4,4,4,4,4,4,4,4,4,4
NEIGHBORHOOD,OTHER,OTHER,OTHER,OTHER,UPPER EAST SIDE (59-79),UPPER EAST SIDE (79-96),OTHER,OTHER,OTHER,OTHER,...,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER
BUILDING_CLASS_CATEGORY,13 CONDOS - ELEVATOR APARTMENTS,21 OFFICE BUILDINGS,21 OFFICE BUILDINGS,13 CONDOS - ELEVATOR APARTMENTS,15 CONDOS - 2-10 UNIT RESIDENTIAL,07 RENTALS - WALKUP APARTMENTS,07 RENTALS - WALKUP APARTMENTS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,...,02 TWO FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,02 TWO FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,09 COOPS - WALKUP APARTMENTS,10 COOPS - ELEVATOR APARTMENTS,02 TWO FAMILY DWELLINGS,12 CONDOS - WALKUP APARTMENTS
TAX_CLASS_AT_PRESENT,2,4,4,2,2C,2B,2B,1,1,1,...,1,1,1,1,1,1,2,2,1,2
BLOCK,716,812,839,592,1379,1551,1891,4090,4120,4120,...,10162,11612,11808,12295,12536,10965,169,131,8932,1216
LOT,1246,68,69,1041,1402,131,159,37,18,20,...,52,73,50,23,38,276,29,4,18,1161
BUILDING_CLASS_AT_PRESENT,R4,O5,O5,R4,R1,C4,C4,A1,A5,A5,...,B1,A1,A0,A1,B3,A5,C6,D4,S2,R2
ADDRESS,"447 WEST 18TH STREET, PH12A",144 WEST 37TH STREET,40 WEST 38TH STREET,"1 SHERIDAN SQUARE, 8C","20 EAST 65TH STREET, B",354 EAST 89TH STREET,304 WEST 106 STREET,1193 SACKET AVENUE,1215 VAN NEST AVENUE,1211 VAN NEST AVENUE,...,104-59 164TH STREET,10919 132ND STREET,135-24 122ND STREET,134-34 157TH STREET,130-26 176 PLACE,111-17 FRANCIS LEWIS BLVD,"45-14 43RD STREET, 3C","50-05 43RD AVENUE, 3M",91-10 JAMAICA AVE,"61-05 39TH AVENUE, F5"
ZIP_CODE,10011,10018,10018,10014,10065,10128,10025,10461,10461,10461,...,11433,11420,11420,11434,11434,11429,11104,11377,11421,11377
RESIDENTIAL_UNITS,1,0,0,1,1,10,10,1,1,1,...,2,1,1,1,2,1,0,0,2,1


In [64]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,23040.0,4.459116e+03,3.713270e+03,1.0,1341.0,3546.0,6673.75,16350.0
LOT,23040.0,3.539485e+02,6.280253e+02,1.0,22.0,49.0,375.00,9057.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,23039.0,1.079119e+04,1.103414e+03,0.0,10306.0,11211.0,11360.00,11697.0
RESIDENTIAL_UNITS,23039.0,1.827423e+00,1.009043e+01,0.0,0.0,1.0,2.00,750.0
COMMERCIAL_UNITS,23039.0,2.865142e-01,5.778224e+00,-148.0,0.0,0.0,0.00,570.0
TOTAL_UNITS,23039.0,2.320891e+00,1.207055e+01,0.0,1.0,1.0,2.00,755.0
GROSS_SQUARE_FEET,23039.0,3.508555e+03,2.218610e+04,0.0,494.0,1356.0,2280.00,1303935.0
YEAR_BUILT,23005.0,1.819672e+03,4.883766e+02,0.0,1920.0,1940.0,1965.00,2019.0
TAX_CLASS_AT_TIME_OF_SALE,23040.0,1.624479e+00,8.038978e-01,1.0,1.0,1.0,2.00,4.0


In [19]:
key = ['ZIP_CODE','LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT']
replace= [0.0]
df[key] = df[key].replace( replace,np.NaN, regex=True)

In [20]:
cols_mean = ['ZIP_CODE','LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'SALE_PRICE']

for column in cols_mean:
    df[column].fillna(df[column].mean(), inplace=True)

In [21]:
key = ['ZIP_CODE', 'BLOCK', 'LOT', 'TAX_CLASS_AT_TIME_OF_SALE']
df[key] = df[key].astype(str)

In [22]:
from datetime import datetime
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])
train = df[df['SALE_DATE']<='2019-03-31']
test = df[df['SALE_DATE']> '2019-03-31']

In [23]:
train.T.head(19)

,0,1,2,3,4,5,6,7,8,9,...,18157,18158,18159,18160,18161,18162,18163,18164,18165,18166
BOROUGH,1,1,1,1,1,1,1,2,2,2,...,3,3,3,4,4,4,4,4,4,5
NEIGHBORHOOD,OTHER,OTHER,OTHER,OTHER,UPPER EAST SIDE (59-79),UPPER EAST SIDE (79-96),OTHER,OTHER,OTHER,OTHER,...,OTHER,OTHER,OTHER,ASTORIA,OTHER,OTHER,FLUSHING-NORTH,OTHER,OTHER,OTHER
BUILDING_CLASS_CATEGORY,13 CONDOS - ELEVATOR APARTMENTS,21 OFFICE BUILDINGS,21 OFFICE BUILDINGS,13 CONDOS - ELEVATOR APARTMENTS,15 CONDOS - 2-10 UNIT RESIDENTIAL,07 RENTALS - WALKUP APARTMENTS,07 RENTALS - WALKUP APARTMENTS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,...,13 CONDOS - ELEVATOR APARTMENTS,03 THREE FAMILY DWELLINGS,02 TWO FAMILY DWELLINGS,02 TWO FAMILY DWELLINGS,14 RENTALS - 4-10 UNIT,02 TWO FAMILY DWELLINGS,13 CONDOS - ELEVATOR APARTMENTS,02 TWO FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,02 TWO FAMILY DWELLINGS
TAX_CLASS_AT_PRESENT,2,4,4,2,2C,2B,2B,1,1,1,...,2,1,1,1,2A,1,2,1,1,1
BLOCK,716,812,839,592,1379,1551,1891,4090,4120,4120,...,8720,7806,6708,723,1723,15763,5137,9854,8133,4960
LOT,1246,68,69,1041,1402,131,159,37,18,20,...,1551,29,11,28,1,13,1243,3,24,16
BUILDING_CLASS_AT_PRESENT,R4,O5,O5,R4,R1,C4,C4,A1,A5,A5,...,R4,C0,B9,B1,S9,B2,R4,S2,A1,B9
ADDRESS,"447 WEST 18TH STREET, PH12A",144 WEST 37TH STREET,40 WEST 38TH STREET,"1 SHERIDAN SQUARE, 8C","20 EAST 65TH STREET, B",354 EAST 89TH STREET,304 WEST 106 STREET,1193 SACKET AVENUE,1215 VAN NEST AVENUE,1211 VAN NEST AVENUE,...,"40 OCEANA DRIVE WEST, 4D",1107 EAST 58TH STREET,908 EAST 16TH STREET,3225 47TH STREET,108-02 NORTHERN BOULEVARD,23-72 BROOKHAVEN AVENUE,"138-35 ELDER AVENUE, 4D",84-45 164TH STREET,43-11 LITTLE NECK PARKWAY,3270 AMBOY ROAD
ZIP_CODE,10011.0,10018.0,10018.0,10014.0,10065.0,10128.0,10025.0,10461.0,10461.0,10461.0,...,11235.0,11234.0,11230.0,11103.0,11368.0,11691.0,11355.0,11432.0,11363.0,10306.0
RESIDENTIAL_UNITS,1,0,0,1,1,10,10,1,1,1,...,1,3,2,2,4,2,1,2,1,2


In [24]:
test.T.head(19)

,18167,18168,18169,18170,18171,18172,18173,18174,18175,18176,...,23030,23031,23032,23033,23034,23035,23036,23037,23038,23039
BOROUGH,1,1,1,1,1,1,1,1,1,1,...,4,4,4,4,4,4,4,4,4,4
NEIGHBORHOOD,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,GRAMERCY,GRAMERCY,GRAMERCY,...,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER
BUILDING_CLASS_CATEGORY,13 CONDOS - ELEVATOR APARTMENTS,13 CONDOS - ELEVATOR APARTMENTS,13 CONDOS - ELEVATOR APARTMENTS,17 CONDO COOPS,21 OFFICE BUILDINGS,15 CONDOS - 2-10 UNIT RESIDENTIAL,13 CONDOS - ELEVATOR APARTMENTS,10 COOPS - ELEVATOR APARTMENTS,13 CONDOS - ELEVATOR APARTMENTS,13 CONDOS - ELEVATOR APARTMENTS,...,02 TWO FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,02 TWO FAMILY DWELLINGS,01 ONE FAMILY DWELLINGS,09 COOPS - WALKUP APARTMENTS,10 COOPS - ELEVATOR APARTMENTS,02 TWO FAMILY DWELLINGS,12 CONDOS - WALKUP APARTMENTS
TAX_CLASS_AT_PRESENT,2,2,2,2,4,2C,2,2,2,2,...,1,1,1,1,1,1,2,2,1,2
BLOCK,695,767,767,721,746,145,31,880,878,878,...,10162,11612,11808,12295,12536,10965,169,131,8932,1216
LOT,1562,1609,1612,1008,64,1203,1050,42,1010,1059,...,52,73,50,23,38,276,29,4,18,1161
BUILDING_CLASS_AT_PRESENT,R4,R4,R4,R9,O8,R1,R4,D4,R4,R4,...,B1,A1,A0,A1,B3,A5,C6,D4,S2,R2
ADDRESS,"555 WEST 23RD STREET, N11E","221 WEST 17TH STREET, 6A","221 WEST 17TH STREET, PH3","465 WEST 23RD STREET, RCU1",340 WEST 23RD STREET,"109 READE STREET, D","75 WALL STREET, 21I","330 THIRD AVENUE, 18K","121 EAST 22ND STREET, N204","121 EAST 22ND STREET, N804",...,104-59 164TH STREET,10919 132ND STREET,135-24 122ND STREET,134-34 157TH STREET,130-26 176 PLACE,111-17 FRANCIS LEWIS BLVD,"45-14 43RD STREET, 3C","50-05 43RD AVENUE, 3M",91-10 JAMAICA AVE,"61-05 39TH AVENUE, F5"
ZIP_CODE,10011.0,10011.0,10011.0,10011.0,10011.0,10013.0,10005.0,10010.0,10010.0,10010.0,...,11433.0,11420.0,11420.0,11434.0,11434.0,11429.0,11104.0,11377.0,11421.0,11377.0
RESIDENTIAL_UNITS,1,1,1,0,3,1,1,0,1,1,...,2,1,1,1,2,1,0,0,2,1


In [25]:
train.select_dtypes(exclude='number').describe().T.sort_values(by='unique')

,count,unique,top,freq,first,last
TAX_CLASS_AT_TIME_OF_SALE,18167,3,1,9473,NaN,NaN
BOROUGH,18167,5,4,5883,NaN,NaN
TAX_CLASS_AT_PRESENT,18167,10,1,8911,NaN,NaN
NEIGHBORHOOD,18167,11,OTHER,15058,NaN,NaN
BUILDING_CLASS_CATEGORY,18167,43,01 ONE FAMILY DWELLINGS,4094,NaN,NaN
SALE_DATE,18167,90,2019-01-24 00:00:00,480,2019-01-01 00:00:00,2019-03-31 00:00:00
BUILDING_CLASS_AT_PRESENT,18167,140,D4,2640,NaN,NaN
BUILDING_CLASS_AT_TIME_OF_SALE,18167,140,D4,2640,NaN,NaN
ZIP_CODE,18167,184,10314.0,296,NaN,NaN
LOT,18167,1326,1,855,NaN,NaN


In [26]:
train.groupby('BOROUGH')['SALE_PRICE'].mean()

BOROUGH
1    3.200017e+06
2    5.736442e+05
3    8.495242e+05
4    7.335611e+05
5    3.947132e+05
Name: SALE_PRICE, dtype: float64

In [27]:
target = 'SALE_PRICE'
high_cardinality = ['ADDRESS', 'BLOCK', 'LOT', 'SALE_DATE']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [28]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [29]:
X_train.head(20)

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,...,BUILDING_CLASS_AT_TIME_OF_SALE_RA,BUILDING_CLASS_AT_TIME_OF_SALE_G4,BUILDING_CLASS_AT_TIME_OF_SALE_V2,BUILDING_CLASS_AT_TIME_OF_SALE_H2,BUILDING_CLASS_AT_TIME_OF_SALE_P9,BUILDING_CLASS_AT_TIME_OF_SALE_W4,BUILDING_CLASS_AT_TIME_OF_SALE_W3,BUILDING_CLASS_AT_TIME_OF_SALE_I7,BUILDING_CLASS_AT_TIME_OF_SALE_H4,BUILDING_CLASS_AT_TIME_OF_SALE_K6
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
from sklearn.feature_selection import f_regression, SelectKBest


selector = SelectKBest(score_func=f_regression, k=54)

# IMPORTANT!
# .fit_transform on the train set
# .transform on test set
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape, X_test_selected.shape

((18167, 54), (4873, 54))

In [31]:
# Which features were selected?
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)
    
print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
BOROUGH_1
BOROUGH_4
BUILDING_CLASS_CATEGORY_21 OFFICE BUILDINGS
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS
BUILDING_CLASS_CATEGORY_32 HOSPITAL AND HEALTH FACILITIES
BUILDING_CLASS_CATEGORY_33 EDUCATIONAL FACILITIES
BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS
BUILDING_CLASS_CATEGORY_22 STORE BUILDINGS
BUILDING_CLASS_CATEGORY_30 WAREHOUSES
BUILDING_CLASS_CATEGORY_25 LUXURY HOTELS
BUILDING_CLASS_CATEGORY_28 COMMERCIAL CONDOS
TAX_CLASS_AT_PRESENT_4
TAX_CLASS_AT_PRESENT_1
BUILDING_CLASS_AT_PRESENT_D6
BUILDING_CLASS_AT_PRESENT_E1
BUILDING_CLASS_AT_PRESENT_D1
BUILDING_CLASS_AT_PRESENT_O4
BUILDING_CLASS_AT_PRESENT_E9
BUILDING_CLASS_AT_PRESENT_O6
BUILDING_CLASS_AT_PRESENT_H1
BUILDING_CLASS_AT_PRESENT_RR
BUILDING_CLASS_AT_PRESENT_D8
BUILDING_CLASS_AT_PRESENT_W8
BUILDING_CLASS_AT_PRESENT_R5
BUILDING_CLASS_AT_PRESENT_K3
BUILDING_CLASS_AT_PRESENT_I4
BUILDING_CLASS_AT_PRESENT_H2
ZIP_CODE_10018.0
ZIP_CODE_10022.0
ZIP

In [43]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

numeric = selected_names

test_scaled = X_test[numeric]
train_scaled = X_train[numeric]

In [44]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_scaled)
X_test_scaled = scaler.transform(test_scaled)

In [47]:
X_train_scaled

array([[ 1.98749497e+00, -6.92037327e-01, -6.65061372e-02, ...,
        -7.41942664e-03, -1.04929426e-02, -1.48400787e-02],
       [ 1.98749497e+00, -6.92037327e-01,  1.50362063e+01, ...,
        -7.41942664e-03, -1.04929426e-02, -1.48400787e-02],
       [ 1.98749497e+00, -6.92037327e-01,  1.50362063e+01, ...,
        -7.41942664e-03, -1.04929426e-02, -1.48400787e-02],
       ...,
       [-5.03145928e-01,  1.44500876e+00, -6.65061372e-02, ...,
        -7.41942664e-03, -1.04929426e-02, -1.48400787e-02],
       [-5.03145928e-01,  1.44500876e+00, -6.65061372e-02, ...,
        -7.41942664e-03, -1.04929426e-02, -1.48400787e-02],
       [-5.03145928e-01, -6.92037327e-01, -6.65061372e-02, ...,
        -7.41942664e-03, -1.04929426e-02, -1.48400787e-02]])

In [48]:
X_test_scaled

array([[ 1.98749497, -0.69203733, -0.06650614, ..., -0.00741943,
        -0.01049294, -0.01484008],
       [ 1.98749497, -0.69203733, -0.06650614, ..., -0.00741943,
        -0.01049294, -0.01484008],
       [ 1.98749497, -0.69203733, -0.06650614, ..., -0.00741943,
        -0.01049294, -0.01484008],
       ...,
       [-0.50314593,  1.44500876, -0.06650614, ..., -0.00741943,
        -0.01049294, -0.01484008],
       [-0.50314593,  1.44500876, -0.06650614, ..., -0.00741943,
        -0.01049294, -0.01484008],
       [-0.50314593,  1.44500876, -0.06650614, ..., -0.00741943,
        -0.01049294, -0.01484008]])

In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [54]:
lin_reg = LinearRegression().fit(train_scaled,y_train)
mean_squared_error(y_train, lin_reg.predict(X_train_scaled))

7.550480282855558e+17

In [68]:
lin_reg = LinearRegression().fit(train_scaled,y_train)
mean_squared_error(y_test, lin_reg.predict(X_test_scaled))

9.039008708050002e+16

In [70]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge().fit(X_train, y_train)
mean_squared_error(y_train, ridge_reg.predict(X_train))

38011039971221.82